In [196]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from keras.models import load_model
import numpy as np 
import pandas as pd 
import os
import glob
import cv2
import random
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import regularizers
from keras.callbacks import CSVLogger
#from livelossplot import PlotLossesKeras
import os
import numpy as np
#from imgaug import augmenters as iaa
#import cv2
from keras.layers.normalization import BatchNormalization
#import seaborn as sns
import pandas as pd 
from keras import initializers
from keras import optimizers
import keras.backend as K
import tensorflow as tf
import operator

In [197]:

val_all_path = '../input/validation-data/val/all'
val_hem_path = '../input/validation-data/val/hem' 

In [198]:
val_all_list = os.listdir(val_all_path)
#val_all_list.sort()

val_hem_list = os.listdir(val_hem_path)          
#val_hem_list.sort()

In [199]:
print('val/all: ', len(val_all_list))
print('val/hem :', len(val_hem_list))

val/all:  1219
val/hem : 648


In [200]:
val_all_batch = np.zeros((len(val_all_list), 210, 210, 3), dtype=np.uint8)
val_hem_batch = np.zeros((len(val_hem_list), 210, 210, 3), dtype=np.uint8) 

In [201]:
print(val_all_batch.shape, val_hem_batch.shape)

(1219, 210, 210, 3) (648, 210, 210, 3)


In [202]:
def Read_n_Crop(list_data, batch, path):
    i=0
    for x in list_data:
        image = cv2.imread(os.path.join(path, x))
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image = crop_center(image, (210,210,3))
        batch[i] = image
        i+=1
    
    print(type(batch), batch.shape, batch.dtype, batch[0].shape, batch[0].dtype)
    return batch

In [203]:
def crop_center(img, bounding):
    start = tuple(map(lambda a, da: a//2-da//2, img.shape, bounding))
    end = tuple(map(operator.add, start, bounding))
    slices = tuple(map(slice, start, end))
    return img[slices]

In [204]:
parasite_images=Read_n_Crop(val_all_list, val_all_batch, val_all_path)
uninf_images=Read_n_Crop(val_hem_list, val_hem_batch, val_hem_path) 

<class 'numpy.ndarray'> (1219, 210, 210, 3) uint8 (210, 210, 3) uint8
<class 'numpy.ndarray'> (648, 210, 210, 3) uint8 (210, 210, 3) uint8


In [205]:
para_label = np.array([0 for _ in range(len(parasite_images))])
uninf_label = np.array([1 for _ in range(len(uninf_images))])

para_label.shape, uninf_label.shape

((1219,), (648,))

In [206]:
x_all = np.concatenate((parasite_images, uninf_images), axis=0)
y_all = np.concatenate((para_label, uninf_label), axis=0)
print(x_all.shape, y_all.shape)

(1867, 210, 210, 3) (1867,)


In [207]:
model = None
model = load_model("../input/modelsforleukemia/Models for 10x Aug/10x_rmsp_baseline_vgg.h5", compile = False)
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 210, 210, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 210, 210, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 210, 210, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 105, 105, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 105, 105, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 105, 105, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 52, 52, 128)       0     

In [208]:
x_all=x_all/255.0
# Make predictions using trained model
y_pred = model.predict(x_all, verbose=1)
print("Predictions: ", y_pred.shape)

59/59 [==============================] - 4s 60ms/step
Predictions:  (1867, 1)


In [209]:
y_pred_flat = []
for pred in y_pred:
   if pred > 0.5:
       y_pred_flat.append(1)
   else:
       y_pred_flat.append(0)
y_pred_flat = np.array(y_pred_flat)

In [210]:
from sklearn.metrics import confusion_matrix, classification_report

# Classification report

confusion_mtx = confusion_matrix(y_all, y_pred_flat) 
print(confusion_mtx)
target_names = ['0', '1']
print(classification_report(y_all, y_pred_flat, target_names=target_names, digits=4))

[[   0 1219]
 [   0  648]]
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      1219
           1     0.3471    1.0000    0.5153       648

    accuracy                         0.3471      1867
   macro avg     0.1735    0.5000    0.2577      1867
weighted avg     0.1205    0.3471    0.1789      1867

